In [1]:
from inference_toolbox.model import Model, add_model, delete_model
from inference_toolbox.likelihood import Likelihood
from inference_toolbox.parameter import Parameter
from visualisation_toolbox.domain import Domain
from controllers.sampler import Sampler
from visualisation_toolbox.visualiser import Visualiser
from data_processing.sim_data_processor import SimDataProcessor
from data_processing.raw_data_processor import RawDataProcessor
import numpy as np
import pandas as pd
import os
import jax

os.chdir('/PhD_project/')

jax.config.update("jax_enable_x64", True)


/build/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
report_title = ''

# sim_model = Model('log_gpm_norm').add_fixed_model_param('H', 5).add_fixed_model_param('I_y',0.04).add_fixed_model_param('I_z',0.0016).add_fixed_model_param('Q',3.41e13)
# sim_domain = Domain('cone_from_source_z_limited').add_domain_param('r',1000).add_domain_param('theta',np.pi/8).add_domain_param('source', [0,0,5]).add_domain_param('resolution',20)

# sim_data_processor = SimDataProcessor('F_log_gaus_noise_pn_10', sim_model, sim_domain,noise_dist='gaussian' ,noise_percentage=0.1)

processor_params = {
    'experiments_list': [
        'Exp1',
        'Exp2',
        'Exp3',
        'Exp4',
        'Exp5',
        'Exp6',
        'Exp7',
        'Exp8',
        'Exp9',
        'Exp10',
        'Exp11',
        'Exp12',
    ],
    'meta_data_select': 'GBR_data_summary',
    'input_header': 'Concentration',
    'output_header': 'Concentration',
    'log_output_data':True,
    'gridding': [100,100,25]
}

raw_data_processor = RawDataProcessor('GBR_data', 
                                      'GBR_data_gridded_100_100_25', 
                                      'GBR_processor', 
                                      processor_params=processor_params,
                                      slices = {'y':0, 'z':5})


model = Model('log_gpm_norm').add_fixed_model_param('H', 5).add_fixed_model_param('Q',3.41e13)
# likelihood = Likelihood('gaussian_percentage_error')
likelihood = Likelihood('gaussian')
# I_y_and_I_z = Parameter(name=['I_y','I_z'],prior_select='log_norm', multi_mode=True).add_prior_param('peak',  [[0.22,0.2],[0.16,0.12],[0.11,0.08],[0.08,0.06], [0.06,0.03], [0.04,0.016]] ).add_prior_param('overall_scale', 1)
# I_y = Parameter(name='I_y',prior_select='log_norm', multi_mode=True).add_prior_param('peak',  [0.22,0.16,0.11,0.08,0.06,0.04] ).add_prior_param('overall_scale', 5)
# I_z = Parameter(name='I_z',prior_select='log_norm', multi_mode=True).add_prior_param('peak',  [0.2,0.12,0.08,0.06,0.03,0.016] ).add_prior_param('overall_scale', 5)

I_y = Parameter(name='I_y',prior_select='uniform').add_prior_param('low',  0.0001).add_prior_param('high', 1)
I_z = Parameter(name='I_z',prior_select='uniform').add_prior_param('low',  0.0001).add_prior_param('high', 1)

# I_y_and_I_z = Parameter(name=['I_y','I_z'],prior_select='log_norm').add_prior_param('peak', [0.11,0.11] ).add_prior_param('scale',[[1,0],[0,1]])
# Q = Parameter(name='Q',prior_select='log_norm', order = 13).add_prior_param('peak',  1).add_prior_param('scale', 0.001)
# sigma = Parameter(name='sigma',prior_select='log_norm').add_prior_param('peak',  1).add_prior_param('scale', 1)
# sigma = Parameter(name='sigma',prior_select='uniform').add_prior_param('low',  0.0001).add_prior_param('high', 5)
sigma = Parameter(name='sigma',prior_select='uniform').add_prior_param('low',  0.0001).add_prior_param('high', 5)
error = Parameter(name='error',prior_select='uniform').add_prior_param('low',  0.0001).add_prior_param('high', 1)
# error = Parameter(name='error',prior_select='log_norm').add_prior_param('peak',  0.8).add_prior_param('scale', 0.00001)
# inference_params = pd.Series({'I_y_and_I_z': I_y_and_I_z, 'sigma':sigma})
# inference_params = pd.Series({'I_y_and_I_z': I_y_and_I_z, 'error':error})
# inference_params = pd.Series({'I_y_and_I_z': I_y_and_I_z})
# inference_params = pd.Series({'I_y': I_y, 'I_z': I_z, 'error':error})
inference_params = pd.Series({'I_y': I_y, 'I_z': I_z, 'sigma':sigma})


# sampler = Sampler(inference_params, model, likelihood, sim_data_processor, n_samples = 10000, n_chains=3)
sampler = Sampler(inference_params, model, likelihood, raw_data_processor, n_samples = 10000, n_chains=3)
sampler.sample_all()

visualiser = Visualiser(sampler)
visualiser.get_traceplots()

domain = Domain('cone_from_source_z_limited').add_domain_param('r',1000).add_domain_param('theta',np.pi/8).add_domain_param('source', [0,0,1]).add_domain_param('resolution',40)
domain.add_domain_param('z_slice', 0)
domain.add_domain_param('y_slice', 0)
domain.add_domain_param('x_slice', 100)

# visualiser.show_predictions(domain, 'test', title = 'Concentration of Droplets', plot_type='3D')
# visualiser.show_predictions(domain, 'test', title = 'Concentration of Droplets', plot_type='2D_slice')
visualiser.get_autocorrelations()
visualiser.get_summary()

Q_ref = {
            'vals' : [2.82e13, 3.11e13, 2.89e13, 4.83e13],
            'labels': ['250m','200m','750m','1000m'] 
        }

I_y_and_I_z_ref = {
            'vals' : [[0.22,0.2],[0.16,0.12],[0.11,0.08],[0.08,0.06], [0.06,0.03], [0.04,0.016]],
            'labels': ['A','B','C','D','E','F'] 
        }

I_y_ref = {
            'vals' : [0.22,0.16,0.11,0.08,0.06,0.04],
            'labels': ['A','B','C','D','E','F'] 
        }

I_z_ref = {'vals' : [0.2,0.12,0.08,0.06,0.03,0.016],
            'labels': ['A','B','C','D','E','F'] 
        }

visualiser.plot_prior('sigma', [0.0001,5])
# visualiser.plot_prior('error', [0,1])
# visualiser.plot_prior('I_y_and_I_z', [[0.0001,0.3],[0.0001,0.3]], I_y_and_I_z_ref)

visualiser.plot_prior('I_y', [0.0001,0.5], I_y_ref)
visualiser.plot_prior('I_z', [0.0001,0.5], I_z_ref)

visualiser.plot_posterior('sigma', [0.0001,5])
# visualiser.plot_posterior('error', [0,1])
# visualiser.plot_posterior('I_y_and_I_z', [[0.0001,0.3],[0.0001,0.3]], I_y_and_I_z_ref)

visualiser.plot_posterior('I_y', [0.0001,0.5], I_y_ref)
visualiser.plot_posterior('I_z', [0.0001,0.5], I_z_ref)

visualiser.generate_report(report_title)
visualiser.embed_report()


Data loaded from data/processed_raw_data/GBR_data_gridded_100_100_25


sample: 100%|██████████| 15000/15000 [00:09<00:00, 1591.69it/s, 7 steps of size 7.68e-01. acc. prob=0.91]


Samples saved to /PhD_project/results/inference_results/GBR_data_gridded_100_100_25/general_instances/instance_6
